In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.options.display.max_colwidth = 100
import sqlite3

In [2]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import os
import glob

In [4]:
from tqdm import tqdm

In [5]:
con = sqlite3.connect("impression_1022.db")
cursor = con.cursor()

cursor.execute("SELECT * FROM impressioncontents")
results = cursor.fetchall()

df = pd.read_sql_query("SELECT * FROM impressioncontents", con)

In [6]:
df.head(1)

,id,name,reference1,reference2,target,impression,date,count,num_touches
0,1,hoge,/templates/color/YELLOW.png,/templates/color/RED.png,/templates/Image/targets/sketch_027_000039.png,-17.0,2021-10-10 19:39:06.155428,1,1


In [7]:
df.tail(1)

,id,name,reference1,reference2,target,impression,date,count,num_touches
17407,17418,shuji.fukawa@keio.jp,/templates/color/BLUE.png,/templates/color/BLUE_GREEN.png,/templates/Image/targets/sketch_342_000112.png,50.0,2021-10-22 00:20:18.670564,501,1


In [8]:
df['name'].value_counts().head()

14236koyama@keio.jp      1008
aeol.yy@keio.jp           770
hoge2                     745
shuji.fukawa@keio.jp      668
ryoyayakuruto@keio.jp     636
Name: name, dtype: int64

In [9]:
# df = df.query('num_touches != 0
# 触った回数が0 => 画像がうまく読み込めていない、or 見失った。　ので削除する。
df_0 = df
df = df[df['num_touches'] !=0]

In [10]:
df = df[(df['name']!='hoge2') & (df['name']!='hoge3') & (df['name']!='hoge') & (df['name']!='komei') & (df['name']!='saito')]

In [11]:
df.reset_index(drop=True).head()

,id,name,reference1,reference2,target,impression,date,count,num_touches
0,1065,naoto_yoshida@keio.jp,/templates/color/BLUE_GREEN.png,/templates/color/YELLOW.png,/templates/Image/targets/sketch_007_000051.png,44.0,2021-10-13 17:04:59.234729,1,1
1,1066,naoto_yoshida@keio.jp,/templates/color/BLUE_GREEN.png,/templates/color/YELLOW.png,/templates/Image/targets/i620ye-fn.png,-40.0,2021-10-13 17:05:05.007810,2,1
2,1067,naoto_yoshida@keio.jp,/templates/color/BLUE_GREEN.png,/templates/color/RED.png,/templates/Image/targets/i086sd-fn.png,-38.0,2021-10-13 17:05:10.037438,3,1
3,1068,naoto_yoshida@keio.jp,/templates/color/RED.png,/templates/color/BLUE_GREEN.png,/templates/Image/targets/sketch_327_000113.png,-32.0,2021-10-13 17:05:15.174908,4,1
4,1070,naoto_yoshida@keio.jp,/templates/color/BLUE_GREEN.png,/templates/color/YELLOW.png,/templates/Image/targets/sketch_001_000023.png,-39.0,2021-10-13 17:05:24.343812,5,1


In [12]:
# アノテーションされている回数は？
len(df)

13980

In [13]:
# 何名のユーザーが実験をしているか？
df['name'].nunique()

32

# impressionの値を見て寒色か暖色を分けるcolumnsを作る

In [14]:
df['cool_warm'] = 0

In [15]:
df['cool_warm'] = df['cool_warm'].mask((df['impression']<0)&(df['reference1'].str.contains('YELLOW|RED')), 1)
df['cool_warm'] = df['cool_warm'].mask((df['impression']>0)&(df['reference2'].str.contains('YELLOW|RED')), 1)

In [16]:
# 若干、寒色の方が多い？？
df['cool_warm'].value_counts()

0    7157
1    6823
Name: cool_warm, dtype: int64

# カテゴリーのごとのリストを作る

In [17]:
domain_list = df[df['target'].str.contains('sketch')]['target'].values.tolist()
muct_list = df[df['target'].str.contains('-')]['target'].values.tolist()
tezuka_list = df[(~df['target'].isin(domain_list))&(~df['target'].isin(muct_list))]['target'].values.tolist()

In [18]:
# 数の確認
len(df) , len(domain_list) ,len(muct_list) ,len(tezuka_list) 

(13980, 5550, 5558, 2872)

# 個人ごとに標準化する。

In [19]:
def scale(x):
    res = (x - np.mean(x)) / np.std(x, ddof=1)
    return res

In [20]:
df['impression_scale'] = df.groupby('name')['impression'].transform(scale)

In [21]:
df['impression_scale_abs'] = df['impression_scale'].abs()

In [22]:
df

,id,name,reference1,reference2,target,impression,date,count,num_touches,cool_warm,impression_scale,impression_scale_abs
1064,1065,naoto_yoshida@keio.jp,/templates/color/BLUE_GREEN.png,/templates/color/YELLOW.png,/templates/Image/targets/sketch_007_000051.png,44.0,2021-10-13 17:04:59.234729,1,1,1,2.190201,2.190201
1065,1066,naoto_yoshida@keio.jp,/templates/color/BLUE_GREEN.png,/templates/color/YELLOW.png,/templates/Image/targets/i620ye-fn.png,-40.0,2021-10-13 17:05:05.007810,2,1,0,-2.021663,2.021663
1066,1067,naoto_yoshida@keio.jp,/templates/color/BLUE_GREEN.png,/templates/color/RED.png,/templates/Image/targets/i086sd-fn.png,-38.0,2021-10-13 17:05:10.037438,3,1,0,-1.921380,1.921380
1067,1068,naoto_yoshida@keio.jp,/templates/color/RED.png,/templates/color/BLUE_GREEN.png,/templates/Image/targets/sketch_327_000113.png,-32.0,2021-10-13 17:05:15.174908,4,1,1,-1.620533,1.620533
1069,1070,naoto_yoshida@keio.jp,/templates/color/BLUE_GREEN.png,/templates/color/YELLOW.png,/templates/Image/targets/sketch_001_000023.png,-39.0,2021-10-13 17:05:24.343812,5,1,0,-1.971521,1.971521
...,...,...,...,...,...,...,...,...,...,...,...,...
17401,17412,shuji.fukawa@keio.jp,/templates/color/YELLOW.png,/templates/color/RED.png,/templates/Image/targets/i620yb-fn.png,-39.0,2021-10-22 00:19:53.218762,497,1,1,-0.990478,0.990478
17402,17413,shuji.fukawa@keio.jp,/templates/color/BLUE.png,/templates/color/RED.png,/templates/Image/targets/sketch_262_000547.png,50.0,2021-10-22 00:19:57.410254,498,1,1,1.096490,1.096490
17403,17414,shuji.fukawa@keio.jp,/templates/color/RED.png,/templates/color/BLUE_GREEN.png,/templates/Image/targets/sketch_129_000059.png,-45.0,2021-10-22 00:20:01.474647,499,1,1,-1.131173,1.131173
17404,17415,shuji.fukawa@keio.jp,/templates/color/BLUE_GREEN.png,/templates/color/YELLOW.png,/templates/Image/targets/i223vc-mn.png,50.0,2021-10-22 00:20:05.578262,500,1,1,1.096490,1.096490


9500枚のうちどれほどの画像がアノテーションされているか？

In [23]:
df['target'].nunique() , len(glob.glob('targets/*.png'))

(7319, 9469)

# targetごとに色を当てはめる。
red = 0 , yellow = 1 , blue= 2 , BLUE_GREEN = 3

アノテーション回数とに当てはめ方が変わる。
- １回ならその色にすればよい。
- ２回で両方意見が一致しているのであればその色にすれば良いが、意見が割れている時には個人ごとの正規化が高い方にすれば良い。
- ３回以上であれば、同様に、２回以上と同様のやり方。

それぞれのアノテーション結果に対して、色を割り当てる

In [24]:
df['color'] = 0

In [25]:
# df['color'].mask((df['impression']<0)&(df['reference1'].str.contains('RED')), 0)
df['color'] = df['color'].mask((df['impression']<0)&(df['reference1']=='/templates/color/YELLOW.png'), 1)
df['color'] = df['color'].mask((df['impression']<0)&(df['reference1']=='/templates/color/BLUE.png'), 2)
df['color'] = df['color'].mask((df['impression']<0)&(df['reference1']=='/templates/color/BLUE_GREEN.png'), 3)

df['color'] = df['color'].mask((df['impression']>0)&(df['reference2']=='/templates/color/YELLOW.png'), 1)
df['color'] = df['color'].mask((df['impression']>0)&(df['reference2']=='/templates/color/BLUE.png'), 2)
df['color'] = df['color'].mask((df['impression']>0)&(df['reference2']=='/templates/color/BLUE_GREEN.png'), 3)

In [26]:
df['color'].value_counts()

1    4043
3    3808
2    3341
0    2788
Name: color, dtype: int64

In [27]:
df_target = df.groupby('target')['color'].agg(['count']).reset_index()

In [29]:
# 以下のようにcountのリストを作る or df['target'].value_counts().reset_index() の2通りやり方はある。
count_list_1 = df_target[df_target['count'] == 1]['target'].values.tolist()
count_list_2 = df_target[df_target['count'] == 2]['target'].values.tolist()
count_list_3 = df_target[df_target['count'] == 3]['target'].values.tolist()
count_list_4 = df_target[df_target['count'] == 4]['target'].values.tolist()
count_list_5 = df_target[df_target['count'] >= 5]['target'].values.tolist()

In [31]:
d = pd.DataFrame()
# 画像に対してアノテーションされた色の数で場合分けをする

for i in tqdm(count_list_1):
    d = d.append(df[df['target']==i][['target','color']])

# ２回の場合、絶対値の最大値を撮ればよい。(たまに最大値が同じものがあるので重複してしまう。それはいい印象を与えれていないので削除する。)
for i in tqdm(count_list_2):
    d = d.append(df[df['target']==i][df[df['target']==i]['impression_scale'].abs().max() == df[df['target']==i]['impression_scale'].abs()][['target','color']])
d = d[~d['target'].duplicated()]

# 3回の場合、色の数が３回ならば、絶対値の最大値を撮ればよい。2色以下ならば最頻値を取る。
for i in tqdm(count_list_3):
    if df[df['target']==i]['color'].nunique() == 3:
        d = d.append(df[df['target']==i][df[df['target']==i]['impression_scale'].abs().max() == df[df['target']==i]['impression_scale'].abs()][['target','color']])
    else:
        mode = df[df['target']==i]['color'].mode().values.tolist()[0]
        d = d.append(df[(df['target']==i)&(df['color']== mode)][['target','color']].head(1))    
d = d[~d['target'].duplicated()]

# 4回の場合、色の数が4回ならば、絶対値の最大値を撮ればよい。3色ならば最頻値を取る。2色以下ならば最頻値を取る。
for i in tqdm(count_list_4):
    if df[df['target']==i]['color'].nunique() == 4:
        d = d.append(df[df['target']==i][df[df['target']==i]['impression_scale'].abs().max() == df[df['target']==i]['impression_scale'].abs()][['target','color']])
    elif df[df['target']==i]['color'].nunique() == 3:
        mode = df[df['target']==i]['color'].mode().values.tolist()[0]
        d = d.append(df[(df['target']==i)&(df['color']== mode)][['target','color']].head(1))    
    elif df[df['target']==i]['color'].nunique() <= 2:
        # 絶対値の合計が多い方を選ぶ。
        color_max = df[df['target']==i].groupby('color')['impression_scale_abs'].max().sort_values(ascending=False).reset_index()['color'][0]
        d = d.append(df[(df['target']==i)&(df['color']== color_max)][['target','color']].head(1))
d = d[~d['target'].duplicated()]

# 5回以上の場合、最頻値をとる。
for i in tqdm(count_list_5):
    mode = df[df['target']==i]['color'].mode().values.tolist()[0]
    d = d.append(df[(df['target']==i)&(df['color']== mode)][['target','color']].head(1))
d = d[~d['target'].duplicated()]

100%|██████████| 177/177 [00:00<00:00, 235.04it/s]


In [32]:
len(d) == len(count_list_1)+len(count_list_2)+len(count_list_3)+len(count_list_4)+len(count_list_5)

True

In [39]:
d = d.reset_index(drop=True)

In [40]:
d

,target,color
0,/templates/Image/targets/J・ムラカミ_00004.png,0
1,/templates/Image/targets/NUSTVアナウンサー_00002.png,3
2,/templates/Image/targets/NUSTVアナウンサー_00004.png,3
3,/templates/Image/targets/i000qe-fn.png,1
4,/templates/Image/targets/i000re-fn.png,3
...,...,...
7314,/templates/Image/targets/東京の夫婦_夫_00003.png,0
7315,/templates/Image/targets/焼野矢八_00052.png,1
7316,/templates/Image/targets/羽澄マリ_00012.png,1
7317,/templates/Image/targets/越前_00005.png,2


In [41]:
d.to_csv('target_color.csv')